In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.datasets as dsets
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
from model_28 import Generator, Discriminator
from IPython.display import clear_output
import numpy as np
import datetime
import time
import os

In [15]:
batch_size = 64

img_transform = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = MNIST(root='./data/MNIST', download=True, train=True, transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [16]:
def cuda(data):
    if torch.cuda.is_available():
        return data.cuda()
    else:
        return data
    
def train(steps = 1000000, batch_size = 64, z_dim = 100):
    # Initialize model
    G = cuda(Generator(batch_size))
    D = cuda(Discriminator(batch_size))
    
    # Make directory for samples and models
    cwd = os.getcwd()
    if not os.path.exists(cwd+'/samples'):
        os.makedirs(cwd+'/samples')
    if not os.path.exists(cwd+'/models'):
        os.makedirs(cwd+'/models')

    # Initialize optimizer with filter, lr and coefficients
    g_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, G.parameters()), 0.0001, [0.0,0.9])
    d_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, D.parameters()), 0.0004, [0.0,0.9])
    
    # Load data
    Iter = iter(dataloader)
    
    # Start timer
    start_time = time.time()
    
    for step in range(steps):
        # ================== Train D ================== #
        D.train(); G.train()
        try:
            real_images,_ = next(Iter)
        except:
            Iter = iter(dataloader)
            real_images,_ = next(Iter)
        
        # Compute loss with real images
        d_out_real,_ = D(cuda(real_images))
        d_loss_real = - torch.mean(d_out_real)
        
        # Compute loss with fake images
        z = cuda(torch.randn(batch_size, z_dim))
        fake_images,_ = G(z)
        d_out_fake,_ = D(fake_images)
        d_loss_fake = d_out_fake.mean()
        
        # Backward + Optimize
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad(); g_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================== Train G ================== #
        # Create random noise
        z = cuda(torch.randn(batch_size, z_dim))
        fake_images,_ = G(z)
        g_out_fake,_ = D(fake_images)
        g_loss_fake = - g_out_fake.mean()
        d_optimizer.zero_grad(); g_optimizer.zero_grad()
        g_loss_fake.backward()
        g_optimizer.step()
        
        # Print out log info
        if (step + 1) % 10 == 0:
            elapsed = time.time() - start_time
            elapsed = str(datetime.timedelta(seconds=elapsed))
            clear_output(wait=True)
            print("Elapsed [{}], G_step [{}/{}], D_step[{}/{}], d_out_real: {:.4f}, "
                  " ave_generator_gamma: {:.4f}".
                  format(elapsed, step + 1, steps, (step + 1),
                         steps, d_loss_real.item(),
                         G.attn.gamma.mean().item()))
        
        # Sample images
        if (step + 1) % (10000) == 0:
            fake_images,_= G(fixed_z)
            save_image(denorm(fake_images.data),
                        os.path.join('./samples', '{}_fake.png'.format(step + 1)))
        
        # Save models
        if (step+1) % (10000) == 0:
            torch.save(G.state_dict(),os.path.join('./models', '{}_G.pth'.format(step + 1)))
            torch.save(D.state_dict(),os.path.join('./models', '{}_D.pth'.format(step + 1)))

In [17]:
train()

Elapsed [0:00:33.572258], G_step [570/1000000], D_step[570/1000000], d_out_real: 545.7120,  ave_generator_gamma: -0.0175


KeyboardInterrupt: 